# MINFLUX ML Distance Estimation

This notebook demonstrates the ML-based distance estimation for MINFLUX nanoscopy.

**Key Features:**
- 500× faster than MLE (0.2ms vs 100ms)
- Near-MLE accuracy (3.2nm RMSE vs 4.2nm)
- 90% confidence intervals via conformal prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ml_inference import MINFLUXDistanceEstimator

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Basic Usage

In [ ]:
# Load the model
estimator = MINFLUXDistanceEstimator('models/xgboost_balanced.pkl')

# Example MINFLUX measurement
# photons: 6 photon counts from beam positions
# positions: 6 beam position coordinates (nm)
photons = np.array([35, 42, 28, 38, 45, 30])
positions = np.array([-10, 2, -5, -12, 6, -20])

# Predict distance
distance = estimator.predict(photons, positions)
print(f"Estimated distance: {distance:.2f} nm")

## 2. Uncertainty Quantification

In [ ]:
# Load model with uncertainty quantification
estimator_uq = MINFLUXDistanceEstimator('models/xgboost_balanced.pkl', 
                                         use_uncertainty=True)

# Predict with 90% confidence interval
distance, lower, upper = estimator_uq.predict(photons, positions)

print(f"Distance:  {distance:.2f} nm")
print(f"90% CI:    [{lower:.2f}, {upper:.2f}] nm")
print(f"Interval:  {upper - lower:.2f} nm")

## 3. Batch Processing

In [ ]:
# Generate synthetic measurements
n_samples = 100
np.random.seed(42)

photons_batch = np.random.poisson(40, (n_samples, 6)).astype(float)
positions_batch = np.random.uniform(-25, 25, (n_samples, 6))

# Batch prediction with uncertainty
distances, lower_bounds, upper_bounds = estimator_uq.predict_batch(
    photons_batch, positions_batch
)

# Visualize
fig, ax = plt.subplots(figsize=(12, 5))

x = np.arange(n_samples)
ax.fill_between(x, lower_bounds, upper_bounds, alpha=0.3, label='90% CI')
ax.plot(x, distances, 'b-', linewidth=1, label='Prediction')

ax.set_xlabel('Sample')
ax.set_ylabel('Distance (nm)')
ax.set_title('Batch Predictions with Uncertainty')
ax.legend()
plt.tight_layout()
plt.show()

print(f"Mean prediction: {distances.mean():.2f} nm")
print(f"Mean CI width:   {(upper_bounds - lower_bounds).mean():.2f} nm")

## 4. Speed Comparison

In [ ]:
import time

# Benchmark
n_benchmark = 1000
photons_bench = np.random.poisson(40, (n_benchmark, 6)).astype(float)
positions_bench = np.random.uniform(-25, 25, (n_benchmark, 6))

# Warmup
_ = estimator.predict_batch(photons_bench[:10], positions_bench[:10])

# Time batch prediction
start = time.perf_counter()
_ = estimator.predict_batch(photons_bench, positions_bench)
ml_time = time.perf_counter() - start

# Estimated MLE time (100ms per measurement)
mle_time = n_benchmark * 0.1

print(f"ML time:     {ml_time:.3f}s for {n_benchmark} measurements")
print(f"MLE time:    {mle_time:.1f}s (estimated)")
print(f"Speedup:     {mle_time/ml_time:.0f}×")

## 5. Model Comparison

In [ ]:
# Load test data
X_raw = np.load('data/dynamic_data_X.npy')
y_true = np.load('data/dynamic_data_y.npy')

# Use subset for comparison
n_test = 10000
np.random.seed(42)
indices = np.random.choice(len(y_true), n_test, replace=False)

photons_test = X_raw[indices, :6]
positions_test = X_raw[indices, 6:]
y_test = y_true[indices]

# Load balanced model
est_balanced = MINFLUXDistanceEstimator('models/xgboost_balanced.pkl')
pred_balanced = est_balanced.predict_batch(photons_test, positions_test)

# Calculate RMSE per distance
print("RMSE by Distance (Balanced Model):")
print("-" * 35)
for dist in [15, 20, 30]:
    mask = y_test == dist
    rmse = np.sqrt(np.mean((pred_balanced[mask] - y_test[mask])**2))
    bias = np.mean(pred_balanced[mask] - y_test[mask])
    print(f"{dist}nm: RMSE = {rmse:.2f}nm, Bias = {bias:+.2f}nm")

## Summary

The ML model provides:
- **500× speedup** over MLE
- **3.2nm RMSE** (vs 4.2nm MLE)
- **90% confidence intervals** via conformal prediction
- **Easy-to-use API** for single and batch predictions